In [3]:
import string

import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danieladdison/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
df = pd.read_csv('spam_ham_dataset.csv')

In [6]:
df

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,2933,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


In [7]:
df['text'] = df['text'].apply(lambda x: x.replace('\r\n', ''))

In [8]:
df

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291this...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001( see att...",0
2,3624,ham,"Subject: neon retreatho ho ho , we ' re around...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springsthis deal is to bo...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ftthe transport vol...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following nomshpl ca...,0
5168,2933,ham,"Subject: calpine daily gas nomination>>julie ,...",0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


In [9]:
df.text.iloc[1]

'Subject: hpl nom for january 9 , 2001( see attached file : hplnol 09 . xls )- hplnol 09 . xls'

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5171 entries, 0 to 5170
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5171 non-null   int64 
 1   label       5171 non-null   object
 2   text        5171 non-null   object
 3   label_num   5171 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 161.7+ KB


In [11]:
stemmer = PorterStemmer()
corpus = []

stopwords_set = set(stopwords.words('english'))


for i in range(0, len(df)):
    text = df['text'].iloc[i].lower()
    text = text.translate(str.maketrans('', '', string.punctuation)).split()
    text = [stemmer.stem(word) for word in text if word not in stopwords_set]
    text = ' '.join(text)
    corpus.append(text)

In [12]:
df.text.iloc[0]


"Subject: enron methanol ; meter # : 988291this is a follow up to the note i gave you on monday , 4 / 3 / 00 { preliminaryflow data provided by daren } .please override pop ' s daily volume { presently zero } to reflect dailyactivity you can obtain from gas control .this change is needed asap for economics purposes ."

In [13]:
corpus[0]

'subject enron methanol meter 988291thi follow note gave monday 4 3 00 preliminaryflow data provid daren pleas overrid pop daili volum present zero reflect dailyact obtain ga control chang need asap econom purpos'

In [14]:
vectorizer = CountVectorizer()

x = vectorizer.fit_transform(corpus).toarray()
y = df.label_num

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [15]:
clf = RandomForestClassifier(n_jobs=-1)

clf.fit(x_train, y_train)

RandomForestClassifier(n_jobs=-1)

In [16]:
clf.score(x_test,y_test)

0.9671497584541063

In [17]:
email_to_classify = df.text.values[15]

In [18]:
email_to_classify

"Subject: underpriced issue with high return on equitystock report .dont sieep on this stock ! this is a hot one !company : gaming transactions inc .stock symbol : ggtscurrentiy trading at : o . 30rating : strong purchasenear - term target : 0 . 45long - term target : 1 . oobreaking news for ggts :gaming transactions inc . ( ggts ) , a | eading provider of online gamingporta | management is pleased to announce that it has launched itsproprietary gaming portal ( k e n o . com ) furthermore , the company has begun anintensive marketing campaign to support the | aunch and establish itselfas the | eader in the online gaming industry .( k e n o . c o m ) is an oniine games destination where people piaypopular casino style games to win real money . the foundation of the site isan online version of keno . the game of keno uses 80 balls numbered 1thru 8 o . every game , the house draws 20 bails at random and displaystheir numbers on screens ( called keno boards ) located on the website . theob

In [19]:
email_text = email_to_classify.lower().translate(str.maketrans('', '', string.punctuation)).split()
email_text = [stemmer.stem(word) for word in text if word not in stopwords_set]
email_text = ' '.join(email_text)

email_corpus = [email_text]

x_email = vectorizer.transform(email_corpus)

In [20]:
clf.predict(x_email)

array([1])

In [21]:
df.label_num.iloc[15]

1